<a href="https://colab.research.google.com/github/raviS123-dot/Custom-training-loop-Tensorflow/blob/main/custom_training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers 
import numpy as np

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
batch_size = 64
'''
Batch size is a term used in machine learning and refers to the number of training examples utilized in one iteration.

batch mode: where the batch size is equal to the total dataset thus making the iteration and epoch values equivalent.

mini-batch mode: where the batch size is greater than one but less than the total dataset size. Usually, a number that
can be divided into the total dataset size.

stochastic mode: where the batch size is equal to one. Therefore the gradient and the neural network parameters are updated after each sample.
'''
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
x_val = x_train[-10000:]
y_val = y_train[-10000:]

x_train = x_train[:-10000]
y_train = y_train[:-10000]


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
print(len(train_dataset))

782


In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
model.compile(optimizer, loss_fn)

In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 135.6865
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.4537
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 1.0489
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 1.5530
Seen so far: 38464 samples
Training acc over epoch: 0.5902
Validation acc: 0.7498
Time taken: 8.58s

Start of epoch 1
Training loss (for one batch) at step 0: 0.6109
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.0584
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 1.0312
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 1.0578
Seen so far: 38464 samples
Training acc over epoch: 0.7595
Validation acc: 0.8257
Time taken: 20.95s


In [ ]:
model.evaluate(x_test, y_test, batch_size)

157/157 [==============================] - 0s 2ms/step - loss: 0.7026


0.7026069760322571

In [ ]:
prediction = model.predict([x_test])

In [ ]:
print(np.argmax(prediction[1]))

2
